In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import os
from IPython.display import display, clear_output
import scipy.optimize as spo
import iminuit
import uproot
import json
import multiprocessing as mp
import time
from scipy.optimize import curve_fit

In [7]:
class track_and_readout:
    
    def __init__(self, pe, angle,file_number_range,main_path,save_path):
        self.angle = angle
        self.pe = pe
        self.file_number_range = file_number_range
        self.main_path = main_path
        self.save_path = save_path
        print("Initializing",self.pe,"pe, at angle",self.angle)
        
        self.area_collection = self.load_readout_data(self)
        self.track_data,self.total_events = self.load_track_data_old(self)
        self.e_stat,self.Edep_dist,self.e_ID = self.electron_count_old(self)
        self.track_data,self.total_events = self.load_track_data(self)
        self.e_stat,self.Edep_dist,self.e_ID = self.electron_count(self)
        
    def load_readout_data(self,arg):
        collection = []
        
        for num in self.file_number_range:
            path = self.main_path + "SiPM/"
            file_name = path + "SiPM_readout_" + str(self.pe) + "_" + str(self.angle) + "_run_" + str(num) +".txt"
            #print("reading " + file_name)
            #data = pd.read_json(file_name,lines=True)
            #data_column = np.array(data['voltages']) #.to_numpy()
            #for i in range(len(data_column)):
            #    collection.append(np.trapz(data_column[i]))
            with open(file_name) as f:
                for jsonObj in f:
                    data = json.loads(jsonObj)
                    try:
                        data_column = np.array(data['voltages'])
                        #for i in range(len(data_column)):
                        collection.append(np.trapz(data_column))
                    except: pass
        filename = (f'_{self.pe}PE_{self.angle}angle')
        path = self.save_path + "electron_counts/"
        #np.save(path+'electron_count_area'+filename+'.npy',collection)
        return collection    
   
    def electron_count_old(self,arg):  #counting non-returning backscattered electron directly, and their energy deposited
        print('counting electrons with old method...')
        t_start = time.time()
        non_returning = 0     # electrons getting back-scattered to somewhere else
        non_returning_array = []

        returning_electron = 0     # electrons that end up in the scintillator
        returning_electron_array = []
        
        straight_electron = 0
        straight_electron_array = []
        
        never_electron = 0    # which never goes into the scintillator
        never_array = [] 
        
        non_returning_event=[]
        returning_electron_event=[]
        straight_event = []
        never_event=[]
        
        sc_vol = 15
        electron_select = (self.track_data['Parent_ID']==0)
        
        #non_ev = []
        #mask_non = (sc_vol in temp['Volume'].values) & (temp['Volume'].values[-1] != sc_vol)
        #non_ev.append(temp['Event_ID'][electrons_ID & mask_non])
        #print('new list of non-returning',non_ev)
        
        for i in range(0,int(max(self.track_data["Event_ID"])+1)):
            event_select = self.track_data['Event_ID'] == i
            scintillator = self.track_data['Volume'] == sc_vol
            temp = self.track_data[event_select & electron_select]
            for j in range (int(min(temp["Track_ID"])),int(max(temp["Track_ID"])+1)):
                #electrons_ID = self.track_data['Track_ID']==j
                #each_df = self.track_data[event_select & electron_select & electrons_ID]
                #Edep_df = self.track_data[event_select & electron_select & scintillator & electrons_ID]
                electrons_ID = temp['Track_ID']==j
                
                each_df = temp[(electrons_ID)]
                Edep_df = temp[(electrons_ID)&(temp['Volume']==sc_vol)]

                energy = Edep_df["DE"].sum()
                if sc_vol in each_df['Volume'].values and (each_df['Volume'].values)[-1] != sc_vol: #non-returning
                    non_returning += 1
                    non_returning_array.append(energy)
                    non_returning_event.append(i)
                
                flag = 0
                #contains returning and straight electron!
                if sc_vol in each_df['Volume'].values and (each_df['Volume'].values)[-1] == sc_vol:
                    vol_array = each_df['Volume'].values
                    # find the flag (1 for straight, 0 for returning)
                    idx = np.array(np.where(vol_array==sc_vol)[0])
                    maxi, mini = np.max(idx), np.min(idx)
                    if np.sum(idx) == (len(idx)/2)*(mini + maxi): flag = 0
                    else: flag = 1
                    
                    #for k in range(len(vol_array)-1):
                    #    if vol_array[k] == sc_vol and vol_array[k+1]!=sc_vol:
                    #        flag=1
                    
                    if flag == 0:
                        straight_electron += 1
                        straight_electron_array.append(energy)
                        straight_event.append(i)
                    else:
                        returning_electron += 1
                        returning_electron_array.append(energy)
                        returning_electron_event.append(i)
                        
                if not sc_vol in each_df['Volume'].values:
                    never_electron +=1
                    never_array.append(energy)
                    never_event.append(i)
        
        straight_event = list(set(straight_event))
        non_returning_event=list(set(non_returning_event))
        returning_electron_event=list(set(returning_electron_event))
        never_event=list(set(never_event))
        
        #print('old list of non-returning',non_returning_event)
        print(f'time to count electrons {time.time() - t_start:.2f}')
        print('straight event', len(straight_event))
        print('non returning event', len(non_returning_event))
        print('returning electron event', len(returning_electron_event))
        print('never event', len(never_event))
        elcount = np.array([straight_electron,returning_electron,non_returning,never_electron])
        elcount_event = [straight_event,returning_electron_event,non_returning_event,never_event]
        elcount_energy = [straight_electron_array,returning_electron_array,non_returning_array,never_array]
        path = self.save_path + "electron_counts/"
        if not os.path.exists(path):
            os.makedirs(path)
        filename = (f'_{self.pe}PE_{self.angle}angle')
        #np.save(path+'electron_count'+filename+'.npy',elcount)
        #np.save(path+'electron_count_event'+filename+'.npy',elcount_event)
        #np.save(path+'electron_count_energy'+filename+'.npy',elcount_energy)
        return elcount, elcount_event, elcount_energy
    
    def load_track_data_old(self,arg):
        t_start = time.time()
        all_loaded_data = pd.DataFrame(columns=["Event_ID","Parent_ID","Track_ID","Particle","X","Y","Z","Time","KE","DE","Volume"])
        total_len = 0
        
        for fnum, num in enumerate(self.file_number_range):
            path = self.main_path + "tracking/"
            file_name = path + str(self.pe)+"_" + str(self.angle)+"_track_" + str(num) + ".root"
            
            #print("reading " + file_name)#display("reading " + os.path.join(r,filename))
            
            file = uproot.open(file_name)
            tree = file["ntuple/ABALONE"]
            df = pd.DataFrame(columns=["Event_ID","Parent_ID","Track_ID","Particle","X","Y","Z","Time","KE","DE","Volume"])
            df["Event_ID"] = (tree.array("Event_ID")).astype(int)+int(total_len)
            df["Parent_ID"] = (tree.array("Parent_ID")).astype(int)
            df["Track_ID"] = (tree.array("Track_ID")).astype(int)
            df["Particle"] = tree.array("Particle")
            df["X"] = tree.array("X")
            df["Y"] = tree.array("Y")
            df["Z"] = tree.array("Z")
            df["Time"] = tree.array("Time")
            df["KE"] = tree.array("KE")
            df["DE"] = tree.array("DE")
            df["Volume"] = (tree.array("Volume")).astype(int)
            df.loc[df["Particle"] == 0 , "Particle"] = "e-"
            df.loc[df["Particle"] == 1 , "Particle"] = "photon"
            frames = [all_loaded_data,df]
            all_loaded_data = pd.concat(frames)
            total_len += len(set(df["Event_ID"].values))
        print(f'time load with old method {time.time() - t_start:.2f}')
        print('OLD DATAFRAME\n',all_loaded_data)
        all_loaded_data["Event_ID"] = pd.to_numeric(all_loaded_data["Event_ID"], downcast='signed')
        return all_loaded_data,total_len
    
    def load_track_data(self,arg):
        t_start = time.time()
        total_len = 0
        ID, Event, Parent, Track, Particle, X, Y, Z, Time, KE, DE, Volume = [], [], [], [],[], [], [], [],[], [], [], []
        df = pd.DataFrame(columns=["Event_ID","Parent_ID","Track_ID",
                                   "Particle","X","Y","Z","Time","KE","DE","Volume"])
        for fnum, num in enumerate(self.file_number_range):
            path = self.main_path + "tracking/"
            file_name = path + str(self.pe)+"_" + str(self.angle)+"_track_" + str(num) + ".root"
            file = uproot.open(file_name)
            tree = file["ntuple/ABALONE"]           
            
            Ev = tree.array("Event_ID")
            i_split = np.zeros(4,dtype=int)
            for ii in range(5):
                if ii != 0: i_split[ii-1] = np.where(Ev==ii)[0][0]
                Event.append(ii+total_len)
            Parent = np.append(Parent,np.split(tree.array("Parent_ID").astype(int),i_split))
            Track = np.append(Track,np.split(tree.array("Track_ID").astype(int),i_split))
            Particle = np.append(Particle,np.split(tree.array("Particle").astype(int),i_split))
            X = np.append(X,np.split(tree.array("X"),i_split))
            Y = np.append(Y,np.split(tree.array("Y"),i_split))
            Z = np.append(Z,np.split(tree.array("Z"),i_split))
            Time = np.append(Time,np.split(tree.array("Time"),i_split))
            KE = np.append(KE,np.split(tree.array("KE"),i_split))
            DE = np.append(DE,np.split(tree.array("DE"),i_split))
            Volume = np.append(Volume,np.split(tree.array("Volume").astype(int),i_split))
            total_len += 5
        df["Event_ID"], df["Parent_ID"] = Event, Parent
        df["Track_ID"], df["Particle"] = Track, Particle
        df["X"], df["Y"], df["Z"], df["Time"] = X, Y, Z, Time
        df["KE"], df["DE"] = KE, DE
        df["Volume"] = Volume
        print(f'\ntime load with new method {time.time() - t_start:.2f}')
        print('NEW DATAFRAME\n',df)
        return df,total_len
    
    def electron_count(self,arg): 
        print('counting electrons with new method...')
        t_start = time.time()
        non_returning = 0     # electrons getting back-scattered to somewhere else
        non_returning_array = []

        returning_electron = 0     # electrons that end up in the scintillator
        returning_electron_array = []
        
        straight_electron = 0
        straight_electron_array = []
        
        never_electron = 0    # which never goes into the scintillator
        never_array = [] 
        
        non_returning_event=[]
        returning_electron_event=[]
        straight_event = []
        never_event=[]
        
        sc_vol = 15
        for i in range(0,len(self.track_data["Event_ID"])):
            prt = self.track_data['Parent_ID'][i]
            trk = self.track_data['Track_ID'][i]
            vol = self.track_data['Volume'][i][prt==0]
            for j in list(set(trk[prt==0])):
                energy = self.track_data["DE"][i][(prt==0) & (trk==j)].sum()
                if sc_vol in vol and vol[-1] != sc_vol:
                    non_returning += 1
                    non_returning_array.append(energy)
                    non_returning_event.append(i)
                if sc_vol in vol and vol[-1] == sc_vol:
                    idx = np.array(np.where(vol==sc_vol)[0])
                    maxi, mini = np.max(idx), np.min(idx)
                    if np.sum(idx) == (len(idx)/2)*(mini + maxi): flag = 0
                    else: flag = 1
                    if flag == 0:
                        straight_electron += 1
                        straight_electron_array.append(energy)
                        straight_event.append(i)
                    else:
                        returning_electron += 1
                        returning_electron_array.append(energy)
                        returning_electron_event.append(i)
                if not sc_vol in vol:
                    never_electron +=1
                    never_array.append(energy)
                    never_event.append(i)
        
        print(f'time to count electrons {time.time() - t_start:.2f}')
        print('straight event', len(straight_event))
        print('non returning event', len(non_returning_event))
        print('returning electron event', len(returning_electron_event))
        print('never event', len(never_event))
        elcount = np.array([straight_electron,returning_electron,non_returning,never_electron])
        elcount_event = [straight_event,returning_electron_event,non_returning_event,never_event]
        elcount_energy = [straight_electron_array,returning_electron_array,non_returning_array,never_array]
        path = self.save_path + "electron_counts/"
        if not os.path.exists(path):
            os.makedirs(path)
        filename = (f'_{self.pe}PE_{self.angle}angle')
        #np.save(path+'electron_count'+filename+'.npy',elcount)
        #np.save(path+'electron_count_event'+filename+'.npy',elcount_event)
        #np.save(path+'electron_count_energy'+filename+'.npy',elcount_energy)
        return elcount, elcount_event, elcount_energy

In [8]:
file_number_range = range(1,100)
main_path = '/home/pieramico/AIUTO/provo/'
save_path = './'
pe1_ang0 = track_and_readout(1,0,file_number_range,main_path,save_path)

Initializing 1 pe, at angle 0
time load with old method 24.52
OLD DATAFRAME
      Event_ID Parent_ID Track_ID Particle         X         Y          Z  \
0           0         0        1       e-  0.043590  0.000000  68.389981   
1           0         0        1       e-  0.106626  0.043526  58.390918   
2           0         0        1       e-  0.116141  0.054944  48.390930   
3           0         0        1       e-  0.114728  0.059304  38.390932   
4           0         0        1       e-  0.107724  0.059730  28.390934   
5           0         0        1       e-  0.098831  0.058398  18.390938   
6           0         0        1       e-  0.092322  0.057983   8.390941   
7           0         0        1       e-  0.092327  0.060845   0.000000   
8           0         0        1       e-  0.092327  0.060845  -0.000005   
9           0         0        1       e-  0.092327  0.060845  -0.000006   
10          0         0        1       e-  0.092328  0.060845  -0.000013   
11         